In [1]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [2]:
dataset = pd.read_csv('2d SDG Dataset.csv', header=0, low_memory=False).reset_index(drop=True)

In [7]:
geonames = dataset[['GeoAreaCode', 'GeoAreaName']].drop_duplicates()

In [8]:
geolocator = Nominatim(user_agent="DAEN-690-Capstone")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
geonames['Location'] =  geonames.GeoAreaName.apply(geocode)
geonames['Point'] = geonames['Location'].apply(lambda loc: tuple(loc.point) if loc else None)

In [11]:
def remove_third_element(point):
    if isinstance(point, tuple):
        return point[:2]
    else:
        return point

pd.set_option('Display.max_rows', None)
pd.set_option('Display.max_colwidth', None)
geoinfo = geonames.drop('Location', axis=1)
geoinfo = geoinfo.dropna(how='any')
geoinfo.Point = geonames.Point.apply(remove_third_element)
geoinfo.to_csv('GeoLocationInfo.csv', index=False)

In [ ]:
geoinfo.dtypes